The goal of this worksheet is to give you a flavor of how you can use a software for eperimental math. The general technique is the following:
* Compute a high-order approximation
* Guess a general formula
* Prove it

The following exercises show you how to use Sage's Ore-Algebra package, as well as to call the Online Encyclopedia of Integer Sequences (OEIS) from Sage.

# Guessing recurrences with Ore_Algebra

Author: Nadia Lafrenière

The Ore Algebra package is however not in Sage by default. To install it, type in your command line (outside of Sage):

    sage -pip install --upgrade --user git+https://github.com/mkauers/ore_algebra.git
(For this to succeed, you need to have Git installed on you computer).


And before we use it, we must import it to our worksheet, so Sage knows about it

In [3]:
from ore_algebra import *

## Linear and non-linear recurrences


We will start with a basic recurrence, to get our hands on it. For example, consider the sequence (that you all know)
$1,1,2,3,5,8,13,21,34,55,...$ 

In [4]:
fib10 = [1,1,2,3,5,8,13,21,34,55]

We will try to guess a recurrence for this sequence :

In [11]:
A = OreAlgebra(ZZ['n'], 'Fn')
GFib = guess(fib10, A)
GFib

-Fn^2 - Fn + 1

What does that mean? Basically, that $F_{n+2}-F_{n+1}-F_n = 0$, or that the $(n+2)$-*nd* term of the sequence is the sum of the $(n+1)$-*st* and the $n$-*th*, which is coherent with our expectations.

*Why does not it mean that $-F_{n+2}-F_{n+1}+F_n = 0$?* Because the way linear recurrences are often expressed is by saying $a_{n+k} + c_1 a_{n+k−1} + c_2 a_{n+k−2} + \ldots + c_k a_n = 0$. This means that the coefficients here would $c_0 = 1$, $c_1 = -1$ and $c_2 = -1$, and the recurrence would be $F_{n+2}-F_{n+1}-F_n = 0$.

Also, for linear recurrences, that means that this is the denominator of the generating function (by the theorem we saw for solving linear recurrences).

#### Use it!

In [11]:
GFib.to_list([0,1], 15)  # 0 and 1 are the first terms, 15 is the number of terms I want to see

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]

In [14]:
# Changing the base condition
GFib.to_list([7,2], 15)

[7, 2, 9, 11, 20, 31, 51, 82, 133, 215, 348, 563, 911, 1474, 2385]

### Catalan numbers

In [36]:
Cat10 = [catalan_number(i) for i in range(10)]; Cat10

[1, 1, 2, 5, 14, 42, 132, 429, 1430, 4862]

In [39]:
Gcat = guess(Cat10, OreAlgebra(ZZ['n'], 'Cn'))

In [40]:
Gcat

-n*Cn^2 + Cn - 1

This means that $-xC(x)^2 + C(x)-1 = 0$, where $C(x)$ is the generating function for the Catalan numbers... But we have not seen that formula yet. Can you prove it?

## A combinatorial problem

Proposed in E2297, Richard Stanley.

Let $L(n)$ be the total number of distinct monomials appearing in the expansion of the determinant of an $n\times n$ symmetric matrix $A = (a_{ij})$. For instance, $L(1) = 1$, $L(2) =3$, and $L(3) = 5$. Show that
$\sum_{n=0}^\infty L(n)\frac{x^n}{n!} = (1 - x)^{-\frac{1}{2}}exp(\frac{1}{2}x + \frac{1}{4}x^2)$,
where $|x| < 1$, and where we define $L(0)=1$.

Compute the first values and use the OEIS.

In [42]:
def Symmetric_symbolic_matrix(n):
    R = PolynomialRing(QQ, n^2, 'x')
    B = Matrix(R, n, n, R.gens())
    C = list(B)
    for i in range(B.nrows()):
        for j in range(i):
            C[i][j] = B[j][i]
    return matrix(C)

In [48]:
Symmetric_symbolic_matrixc_matrix(4)

[ x0  x1  x2  x3]
[ x1  x5  x6  x7]
[ x2  x6 x10 x11]
[ x3  x7 x11 x15]

In [43]:
def L(n):
    return len(Symmetric_symbolic_matrix(n).determinant().monomials())

In [46]:
Symmetric_symbolic_matrix(3).determinant().monomials()

[x2^2*x4, x1*x2*x5, x0*x5^2, x1^2*x8, x0*x4*x8]

In [47]:
first_values_of_Ln = [L(n) for n in range(1, 10)]  # it takes a while
first_values_of_Ln

[1, 2, 5, 17, 73, 388, 2461, 18155, 152531]

In [49]:
oeis(first_values_of_Ln)  # needs internet

0: A002135: Number of terms in a symmetrical determinant: a(n) = n*a(n-1) - (n-1)*(n-2)*a(n-3)/2.

In [51]:
more_values_of_Ln = oeis('A002135')[:25]  # needs internet
more_values_of_Ln

(1,
 1,
 2,
 5,
 17,
 73,
 388,
 2461,
 18155,
 152531,
 1436714,
 14986879,
 171453343,
 2134070335,
 28708008128,
 415017867707,
 6416208498137,
 105630583492969,
 1844908072865290,
 34071573484225549,
 663368639907213281,
 13580208904207073801)

Compute the first values and guess a recurrence.

In [52]:
guess(more_values_of_Ln, OreAlgebra(ZZ['n'], 'Sn'))

2*Sn^3 + (-2*n - 6)*Sn^2 + n^2 + 3*n + 2

The recurrence guessed means, by rearranging the terms, that the number of monomials satisfies
$S(n+3) = 3 S(n+2) + \binom{n+2}{2} S(n)$. This is also on the OEIS (formula 1)

In [54]:
oeis(2135).formulas()  # needs internet

0: E.g.f.: (1-x)^(-1/2)*exp(x/2+x^2/4).
1: a(n+1) = (n+1)*a(n) - binomial(n, 2)*a(n-2). See Comtet.
2: Asymptotics: a(n) ~ sqrt(2)*exp(3/4-n)*n^n*(1+O(1/n)). - _Pietro Majer_, Oct 27 2009
3: E.g.f.: G(0)/sqrt(1-x) where G(k) = 1 + x*(x+2)/(4*(2*k+1) - 4*x*(x+2)*(2*k+1)/(x*(x+2) + 8*(k + 1)/G(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Jan 31 2013
4: a(n) = Sum_{k=0..n} Sum_{i=0..k} binomial(k,i)*binomial(i-1/2,n-k)*(3^(k-i)*n!)/(4^k*k!)*(-1)^(n-k-i). - _Emanuele Munarini_, Aug 25 2017

The proof of the formula does however not require a computer...

References : 
 * Bruno Salvy, *Experimental Mathematical & Computer Algebra; Exercises*. [Online](https://mathexp2018.metelu.net/slides_cours/salvy-exos.pdf)
 * Thierry Monteil, *Basics about ore_algebra package.* [Online](https://mathexp2018.metelu.net/worksheets/guessing/ore_algebra_basics/ore_algebra_basics.ipynb)
 * Manuel Kauers, Maximilian Jaroschek, and Fredrik Johansson. *The package Ore Algebra* [online](https://www3.risc.jku.at/research/combinat/software/ore_algebra/index.php) in SageMath.